In [12]:
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path 
from sklearn.metrics import mean_absolute_error
from tensorflow.keras import layers
import logging
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import matplotlib.pyplot as plt

In [13]:
dataset1 = pd.read_csv("FINAL_DATASET_1.csv")
dataset2 = pd.read_csv("FINAL_DATASET_2.csv")
dataset3 = pd.read_csv("FINAL_DATASET_3.csv")
dataset4 = pd.read_csv("FINAL_DATASET_4.csv")
dataset5 = pd.read_csv("FINAL_DATASET_5.csv")
dataset6 = pd.read_csv("FINAL_DATASET_6.csv")
dataset7 = pd.read_csv("FINAL_DATASET_7.csv")
dataset8 = pd.read_csv("FINAL_DATASET_8.csv")
dataset9 = pd.read_csv("FINAL_DATASET_9.csv")
dataset10 = pd.read_csv("FINAL_DATASET_10.csv")
dataset11 = pd.read_csv("FINAL_DATASET_11.csv")
dataset12 = pd.read_csv("FINAL_DATASET_12.csv")
dataset13 = pd.read_csv("FINAL_DATASET_13.csv")
dataset14 = pd.read_csv("FINAL_DATASET_14.csv")
dataset15 = pd.read_csv("FINAL_DATASET_15.csv")
dataset = pd.concat([dataset3, dataset4, dataset5, dataset6, dataset7, dataset8, dataset9, dataset10, dataset11, dataset13, dataset14])
del dataset1
del dataset2
del dataset3
del dataset4
del dataset5
del dataset6
del dataset7
del dataset8
del dataset9
del dataset10
del dataset11
del dataset12
del dataset13
del dataset14
del dataset15
dataset.shape

(23242, 131)

In [14]:
mean = 0 
std = 0.1
augmented_data = []
for index, row in final_df.iterrows():
    signal_data = row[:128].values
    non_zero_indices = np.nonzero(signal_data)
    noise = np.random.normal(mean, std, non_zero_indices[0].shape)
    augmented_signal_data = signal_data.copy()
    augmented_signal_data[non_zero_indices] += noise
    new_row = np.concatenate([augmented_signal_data, row[128:].values])
    augmented_data.append(new_row)
augmented_df = pd.DataFrame(augmented_data, columns=final_df.columns)
final_df = pd.concat([final_df, augmented_df], ignore_index=True)
del augmented_df
del augmented_data
final_df.shape

NameError: name 'final_df' is not defined

In [5]:
min_scale = 0.8
max_scale = 1.2
augmented_data = []
for index, row in final_df.iterrows():
    signal_data = row[:128].values
    non_zero_indices = np.nonzero(signal_data)
    scale_factor = np.random.uniform(min_scale, max_scale)
    augmented_signal_data = signal_data.copy()
    augmented_signal_data[non_zero_indices] *= scale_factor
    new_row = np.concatenate([augmented_signal_data, row[128:].values])
    augmented_data.append(new_row)
augmented_df = pd.DataFrame(augmented_data, columns=final_df.columns)
final_df = pd.concat([final_df, augmented_df], ignore_index=True)
del augmented_df
del augmented_data
final_df.shape

(2699728, 131)

In [15]:
max_shift = 30
augmented_data = []
for index, row in dataset.iterrows():
    signal_data = row[:128].values
    first_nonzero_idx = np.nonzero(signal_data)[0][0]
    available_space = max_shift - first_nonzero_idx
    for shift_amount in range(1, available_space + 1):
        shifted_data = np.roll(signal_data, shift_amount)
        new_row = np.concatenate([shifted_data, row[128:].values])
        augmented_data.append(new_row)
augmented_df = pd.DataFrame(augmented_data, columns=dataset.columns)
del augmented_data
final_df = pd.concat([dataset, augmented_df], ignore_index=True)
del augmented_df
del dataset
final_df.shape

In [6]:
X = final_df.drop(columns = ["x_position", "amp_threshold", "max_param"])
y = final_df[["x_position"]]
del final_df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=37)
X_train, X_test, y_train, y_test, X_val, y_val = pd.DataFrame(X_train).to_numpy(), pd.DataFrame(X_test).to_numpy(), pd.DataFrame(y_train).to_numpy(), pd.DataFrame(y_test).to_numpy(), pd.DataFrame(X_val).to_numpy(), pd.DataFrame(y_val).to_numpy()
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))
y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))
y_val = np.reshape(y_val, (y_val.shape[0], y_val.shape[1], 1))
del X
del y

In [11]:

model = tf.keras.Sequential()
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=512, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor = 0.1, patience = 2)
with tf.device("GPU"):
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data = (X_test, y_test), verbose = 1, callbacks = [lr_schedule])
    loss = model.evaluate(X_val, y_val)
    model.save("model_june_new.h5")
    print("EVALUATION", loss)

Epoch 1/100
53995/53995 [==============================] - 303s 6ms/step - loss: 1402988.7500 - mae: 521.1533 - val_loss: 1329329.8750 - val_mae: 495.8930 - lr: 0.0100
Epoch 2/100
53995/53995 [==============================] - 302s 6ms/step - loss: 1329650.2500 - mae: 496.0300 - val_loss: 1328547.7500 - val_mae: 492.3795 - lr: 0.0100
Epoch 3/100
53995/53995 [==============================] - 322s 6ms/step - loss: 1329523.5000 - mae: 495.5900 - val_loss: 1328709.1250 - val_mae: 492.7889 - lr: 0.0100
Epoch 4/100
53995/53995 [==============================] - 313s 6ms/step - loss: 1329546.8750 - mae: 495.6167 - val_loss: 1328811.6250 - val_mae: 493.6137 - lr: 0.0100
Epoch 5/100
53995/53995 [==============================] - 307s 6ms/step - loss: 1329457.6250 - mae: 495.3103 - val_loss: 1329206.2500 - val_mae: 493.9182 - lr: 0.0100
Epoch 6/100
53995/53995 [==============================] - 312s 6ms/step - loss: 1328849.2500 - mae: 493.1632 - val_loss: 1329016.7500 - val_mae: 494.5500 - lr:

KeyboardInterrupt: 

In [8]:
model = keras.models.load_model("model_june_new9.h5")
history = model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='mse', metrics=['mae'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data = (X_test, y_test), verbose = 1, callbacks=[early_stopping])

Epoch 1/100
26105/26105 [==============================] - 148s 6ms/step - loss: 6357.9629 - mae: 60.3052 - val_loss: 6910.9370 - val_mae: 63.2117
Epoch 2/100
26105/26105 [==============================] - 147s 6ms/step - loss: 5856.7695 - mae: 58.3915 - val_loss: 6643.2085 - val_mae: 62.0940
Epoch 3/100
26105/26105 [==============================] - 145s 6ms/step - loss: 5630.7783 - mae: 57.4795 - val_loss: 6504.8130 - val_mae: 61.5976
Epoch 4/100
26105/26105 [==============================] - 148s 6ms/step - loss: 5488.0674 - mae: 56.8387 - val_loss: 6469.1763 - val_mae: 61.4441
Epoch 5/100
26105/26105 [==============================] - 148s 6ms/step - loss: 5370.2310 - mae: 56.3314 - val_loss: 6297.6982 - val_mae: 60.7835
Epoch 6/100
26105/26105 [==============================] - 149s 6ms/step - loss: 5271.6538 - mae: 55.8638 - val_loss: 6245.1670 - val_mae: 60.6116
Epoch 7/100
26105/26105 [==============================] - 150s 6ms/step - loss: 5187.3042 - mae: 55.4784 - val_loss: 

KeyboardInterrupt: 

TypeError: 'NoneType' object is not subscriptable